In [3]:
import os
import cv2
import numpy as np
import mediapipe as mp
from math import dist
import pandas as pd

2023-12-08 22:59:19.984325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 22:59:19.984408: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 22:59:20.046817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 22:59:21.762351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
folder=os.path.join("data")

In [5]:
mp_hands=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7, max_num_hands=1)


In [6]:
def distance_calculation(temp:{}):
    res={}
    for i in range(21):
        for j in range(i+1,21):
            temp1='('+str(i)+","+str(j)+")"
            res[temp1]=dist(temp[i],temp[j])
    x=max(res.values())
    for i in res.values():
        i=i/x
    res1=pd.DataFrame(res,index=[0])
    return res1

In [10]:
res={}
for i in range(21):
    for j in range(i+1,21):
        temp1='('+str(i)+","+str(j)+")"
        res[temp1]=[]
res["Target"]=[]
df=pd.DataFrame(res)

for dir in os.listdir(folder):
    for i in range(100):
        x=os.path.join(folder, dir, str(i)+'.jpg')
        img=cv2.imread(x)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results=hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                myHand = results.multi_hand_landmarks[0]
                temp={}
                for id, lm in enumerate(myHand.landmark):
                    temp[id]=[lm.x,lm.y]
                df1=distance_calculation(temp)
                df1["Target"]=chr(int(dir)+65)
                df=pd.concat([df,df1],ignore_index=True)

print(df)

         (0,1)     (0,2)     (0,3)     (0,4)     (0,5)     (0,6)     (0,7)  \
0     0.155956  0.292912  0.391528  0.471278  0.427361  0.519750  0.599439   
1     0.153880  0.290080  0.388680  0.471529  0.425396  0.518414  0.598284   
2     0.154368  0.288942  0.387246  0.468739  0.428465  0.516852  0.596118   
3     0.153351  0.287299  0.384638  0.466048  0.428128  0.516305  0.594306   
4     0.155952  0.289472  0.387022  0.470049  0.428845  0.517669  0.597083   
...        ...       ...       ...       ...       ...       ...       ...   
2585  0.053410  0.102435  0.128383  0.188981  0.348766  0.488495  0.527334   
2586  0.056805  0.107311  0.132715  0.192016  0.354170  0.492760  0.528351   
2587  0.053210  0.100768  0.125383  0.186310  0.346581  0.484797  0.524644   
2588  0.053977  0.101461  0.126443  0.187509  0.347261  0.488085  0.525552   
2589  0.053932  0.101931  0.127136  0.188589  0.350842  0.487841  0.527781   

         (0,8)     (0,9)    (0,10)  ...   (16,18)   (16,19)   (

In [11]:
df.to_csv("data.csv")